In [11]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/fangzhou/data/level2_i_20221202.csv')
df.head()

long = 100
short = 50


a = df['midp'].diff(long) / df['midp'].shift(long)
a = a.dropna()
a.reset_index(drop = True, inplace = True)
df['long return'] = a

b = df['midp'].diff(short) / df['midp'].shift(short)
b = b.dropna()
b.reset_index(drop = True, inplace = True)
df['short return'] = b

col_reserved = ['fac{}'.format(i) for i in range(1, 34)]
col_reserved.append('long return')
col_reserved.append('short return')

new_df = df[col_reserved]
linear_df = new_df[1200:-1200]
#linear_df.fillna(0, inplace = True)
linear_df.reset_index(drop = True, inplace = True)
linear_df

,fac1,fac2,fac3,fac4,fac5,fac6,fac7,fac8,fac9,fac10,...,fac26,fac27,fac28,fac29,fac30,fac31,fac32,fac33,long return,short return
0,0.000018,0.013011,0.999999,0.001871,-0.061324,-0.031032,-0.000112,-0.010945,0.002210,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000209,-0.000312,-0.000325,0.000651,0.000651
1,0.000019,0.001862,0.999990,0.001399,-0.060856,-0.030808,-0.000448,-0.010954,0.002212,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000209,-0.000312,-0.000325,0.000651,0.000651
2,0.000017,-0.005618,-0.999997,0.002332,-0.059300,-0.030743,-0.000449,-0.011102,0.002207,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000211,-0.000312,-0.000325,0.000651,0.000651
3,0.000018,0.001869,0.999990,0.002799,-0.056428,-0.030477,-0.000434,-0.011120,0.002209,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000211,-0.000312,-0.000325,0.000651,0.000651
4,0.000018,0.000000,0.000000,-0.000467,-0.053220,-0.030216,-0.000434,-0.011137,0.002211,-4.0,...,-0.5,0.068966,0.000652,-0.000480,-0.000480,-0.000211,-0.000312,-0.000325,0.000651,0.000651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72994,-0.000033,-0.005181,-0.999997,-0.002582,-0.008557,-0.007308,-0.011443,-0.020865,0.001662,-8.0,...,-0.6,0.000000,0.000219,-0.001025,-0.001025,-0.000104,0.000170,0.000316,-0.000633,0.000000
72995,-0.000032,0.003442,0.999995,-0.001291,-0.008718,-0.007164,-0.011714,-0.020276,0.001666,-8.0,...,-0.6,0.000000,0.000219,-0.001019,-0.001019,-0.000102,0.000169,0.000316,-0.000633,0.000000
72996,-0.000033,-0.003466,-0.499999,-0.001725,-0.008621,-0.007245,-0.011738,-0.019254,0.001663,-8.0,...,-0.6,0.000000,0.000219,-0.001011,-0.001011,-0.000100,0.000164,0.000226,-0.000633,0.000000
72997,-0.000033,0.000000,0.000000,-0.001296,-0.008697,-0.007193,-0.012011,-0.018918,0.001660,-8.0,...,-0.6,0.000000,0.000219,-0.001010,-0.001010,-0.000096,0.000164,0.000226,-0.000633,0.000000


In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 

window_size = 200

predict_long = []
predict_short = []
# this sequence record the R square of validation set
R_square_long = []
R_square_short = []

#limit = len(linear_df)
limit = 20000

for i in range(window_size, limit):
    train_X = linear_df.iloc[i - window_size : i, :-2]
    train_y_long = linear_df.iloc[i - window_size : i, -2]
    train_y_short = linear_df.iloc[i - window_size : i, -1]

    train_X_long, validate_X_long, train_y_long, validate_y_long = train_test_split(train_X, train_y_long, test_size = 0.1, random_state = 11)
    train_X_short, validate_X_short, train_y_short, validate_y_short = train_test_split(train_X, train_y_short, test_size = 0.1, random_state = 11)

    test_X = linear_df.iloc[i:i+1, :-2]
    test_y_long = np.array(linear_df.iloc[i, -2]).reshape(1, -1)
    test_y_short = np.array(linear_df.iloc[i, -1]).reshape(1, -1)

    #model_short = LinearRegression()
    #model_long = LinearRegression()

    model_long = Lasso(alpha = 0.1)
    model_short = Lasso(alpha = 0.1)
    
    scaler_short = StandardScaler()
    scaler_long = StandardScaler()

    scaled_train_X_short = scaler_short.fit_transform(train_X_short)
    scaled_validate_X_short = scaler_short.transform(validate_X_short)
    scaled_test_X_short = scaler_short.transform(test_X)

    scaled_train_X_long = scaler_long.fit_transform(train_X_long)
    scaled_validate_X_long = scaler_long.transform(validate_X_long)
    scaled_test_X_long = scaler_long.transform(test_X)

    model_short.fit(scaled_train_X_short, train_y_short)
    model_long.fit(scaled_train_X_long, train_y_long)

    predict_short.append(model_short.predict(scaled_test_X_short))
    predict_long.append(model_long.predict(scaled_test_X_long))

    R_square_short.append(model_short.score(scaled_validate_X_short, validate_y_short))
    R_square_long.append(model_long.score(scaled_validate_X_long, validate_y_long))

    if i % 1000 == 0:
        print('i = {}'.format(i))
        print('R_square_short = {}'.format(model_short.score(scaled_validate_X_short, validate_y_short)))
        print('R_square_long = {}'.format(model_long.score(scaled_validate_X_long, validate_y_long)))


i = 1000
R_square_short = -0.043182445607869546
R_square_long = -0.18621760593593928


/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

i = 2000
R_square_short = 1.0
R_square_long = 1.0


/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

i = 3000
R_square_short = -0.005516569364720114
R_square_long = -0.0024828273391812594
i = 4000
R_square_short = -0.026334478181878618
R_square_long = -0.1588266591491947
i = 5000
R_square_short = -0.09749797895892542
R_square_long = -0.1560824003685608
i = 6000
R_square_short = -0.003260300097639446
R_square_long = -0.010013823252708498
i = 7000
R_square_short = -0.02522470431404278
R_square_long = -0.0031920979160608987


...

/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

i = 16000
R_square_short = -0.06730200702274725
R_square_long = -0.12836465117722717
i = 17000
R_square_short = -0.0542667209333878
R_square_long = -0.08515364966464989


/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

i = 18000
R_square_short = -0.003755734983647674
R_square_long = -0.0004796369062656236
i = 19000
R_square_short = -0.08475711678012487
R_square_long = -0.04423524741659235


/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 0.000e+00, tolerance: 0.000e+00
  model = cd_fast.enet_coordinate_descent(
/home/fangzhou/anaconda3/envs/futures/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or

In [17]:
predict_short = np.array(predict_short).reshape(-1)
predict_long = np.array(predict_long).reshape(-1)
R_square_short = np.array(R_square_short).reshape(-1)
R_square_long = np.array(R_square_long).reshape(-1)

new_df = pd.DataFrame()
new_df['predict_short'] = predict_short
new_df['predict_long'] = predict_long
new_df['R_square_short'] = R_square_short
new_df['R_square_long'] = R_square_long

new_df['long return'] = linear_df['long return'].iloc[window_size:limit].reset_index(drop = True)
new_df['short return'] = linear_df['short return'].iloc[window_size:limit].reset_index(drop = True)

new_df['predict_long_sign'] = new_df['predict_long'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df['predict_short_sign'] = new_df['predict_short'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)

In [19]:
print(new_df[['predict_short_sign', 'short return']].corr())
print(new_df[['predict_long_sign', 'long return']].corr())
print(new_df[['predict_short', 'short return']].corr())
print(new_df[['predict_long', 'long return']].corr())

                    predict_short_sign  short return
predict_short_sign            1.000000      0.164097
short return                  0.164097      1.000000
                   predict_long_sign  long return
predict_long_sign           1.000000     0.274919
long return                 0.274919     1.000000
               predict_short  short return
predict_short       1.000000      0.200863
short return        0.200863      1.000000
              predict_long  long return
predict_long      1.000000     0.345197
long return       0.345197     1.000000


In [22]:
a = df['midp'].diff(long) / df['midp'].shift(long)
a = a.dropna()
a.reset_index(drop = True, inplace = True)
df['long return'] = a

b = df['midp'].diff(short) / df['midp'].shift(short)
b = b.dropna()
b.reset_index(drop = True, inplace = True)
df['short return'] = b

col_reserved = ['fac{}'.format(i) for i in range(1, 34)]
col_reserved.append('long return')
col_reserved.append('short return')

new_df = df[col_reserved]
linear_df = new_df[1200:-1200]
#linear_df.fillna(0, inplace = True)
linear_df.reset_index(drop = True, inplace = True)

predict_long = []
predict_short = []
# this sequence record the R square of validation set
R_square_long = []
R_square_short = []

for i in range(window_size, limit):
    train_X = linear_df.iloc[i - window_size : i, :-2]
    train_y_long = linear_df.iloc[i - window_size : i, -2]
    train_y_short = linear_df.iloc[i - window_size : i, -1]

    train_X_long, validate_X_long, train_y_long, validate_y_long = train_test_split(train_X, train_y_long, test_size = 0.1, random_state = 11)
    train_X_short, validate_X_short, train_y_short, validate_y_short = train_test_split(train_X, train_y_short, test_size = 0.1, random_state = 11)

    test_X = linear_df.iloc[i:i+1, :-2]
    test_y_long = np.array(linear_df.iloc[i, -2]).reshape(1, -1)
    test_y_short = np.array(linear_df.iloc[i, -1]).reshape(1, -1)

    #model_short = LinearRegression()
    #model_long = LinearRegression()

    model_long = Ridge(alpha = 0.1)
    model_short = Ridge(alpha = 0.1)
    
    scaler_short = StandardScaler()
    scaler_long = StandardScaler()

    scaled_train_X_short = scaler_short.fit_transform(train_X_short)
    scaled_validate_X_short = scaler_short.transform(validate_X_short)
    scaled_test_X_short = scaler_short.transform(test_X)

    scaled_train_X_long = scaler_long.fit_transform(train_X_long)
    scaled_validate_X_long = scaler_long.transform(validate_X_long)
    scaled_test_X_long = scaler_long.transform(test_X)

    model_short.fit(scaled_train_X_short, train_y_short)
    model_long.fit(scaled_train_X_long, train_y_long)

    predict_short.append(model_short.predict(scaled_test_X_short))
    predict_long.append(model_long.predict(scaled_test_X_long))

    R_square_short.append(model_short.score(scaled_validate_X_short, validate_y_short))
    R_square_long.append(model_long.score(scaled_validate_X_long, validate_y_long))

    if i % 1000 == 0:
        print('i = {}'.format(i))
        print('R_square_short = {}'.format(model_short.score(scaled_validate_X_short, validate_y_short)))
        print('R_square_long = {}'.format(model_long.score(scaled_validate_X_long, validate_y_long)))

i = 1000
R_square_short = 0.9044453063290225
R_square_long = 0.9846804840956633
i = 2000
R_square_short = 1.0
R_square_long = 1.0
i = 3000
R_square_short = 0.9463895135136813
R_square_long = 0.9245829422386371
i = 4000
R_square_short = 0.9557086693558853
R_square_long = 0.9879109799898043
i = 5000
R_square_short = 0.9107519857161093
R_square_long = 0.9842940000023646
i = 6000
R_square_short = 0.9564421314698323
R_square_long = 0.7824354360612701
i = 7000
R_square_short = 0.7455607169118488
R_square_long = 0.5751927367719336
i = 8000
R_square_short = 0.9378895172190704
R_square_long = 0.28374775346758274
i = 9000
R_square_short = 0.8331728588593691
R_square_long = 0.7547519105424622
i = 10000
R_square_short = 0.8207290968398998
R_square_long = 0.8665903654777658
i = 11000
R_square_short = 0.7436440773909325
R_square_long = 0.803483932394482
i = 12000
R_square_short = 0.9262403057604741
R_square_long = 0.9994348376478704
i = 13000
R_square_short = 0.9997140396625023
R_square_long = 0.871

In [25]:
predict_short = np.array(predict_short).reshape(-1)
predict_long = np.array(predict_long).reshape(-1)
R_square_short = np.array(R_square_short).reshape(-1)
R_square_long = np.array(R_square_long).reshape(-1)

new_df = pd.DataFrame()
new_df['predict_short'] = predict_short
new_df['predict_long'] = predict_long
new_df['R_square_short'] = R_square_short
new_df['R_square_long'] = R_square_long

new_df['long return'] = linear_df['long return'].iloc[window_size:limit].reset_index(drop = True)
new_df['short return'] = linear_df['short return'].iloc[window_size:limit].reset_index(drop = True)
new_df['predict_long_sign'] = new_df['predict_long'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df['predict_short_sign'] = new_df['predict_short'].apply(lambda x: 1 if x > 0 else 0 if x == 0 else -1)
new_df

,predict_short,predict_long,R_square_short,R_square_long,long return,short return,predict_long_sign,predict_short_sign
0,-0.000700,-0.000406,0.919429,0.781062,-0.00065,-0.00065,-1,-1
1,0.000085,0.000223,0.932403,0.807919,-0.00065,-0.00065,1,1
2,-0.000452,-0.000486,0.896491,0.685345,-0.00065,-0.00065,-1,-1
3,-0.000617,-0.000644,0.926509,0.609772,-0.00065,-0.00065,-1,-1
4,-0.000604,-0.000593,0.941604,0.536002,-0.00065,-0.00065,-1,-1
...,...,...,...,...,...,...,...,...
19795,-0.000628,0.000376,0.673736,0.429648,0.00000,-0.00065,1,-1
19796,-0.000568,0.000158,0.711695,0.569861,0.00000,-0.00065,1,-1
19797,-0.000619,0.000101,0.532723,0.563434,0.00000,-0.00065,1,-1
19798,-0.000596,0.000137,0.087162,0.639261,0.00000,-0.00065,1,-1


In [26]:
print(new_df[['predict_short_sign', 'short return']].corr())
print(new_df[['predict_long_sign', 'long return']].corr())
print(new_df[['predict_short', 'short return']].corr())
print(new_df[['predict_long', 'long return']].corr())

                    predict_short_sign  short return
predict_short_sign            1.000000      0.571921
short return                  0.571921      1.000000
                   predict_long_sign  long return
predict_long_sign           1.000000     0.651021
long return                 0.651021     1.000000
               predict_short  short return
predict_short       1.000000      0.812224
short return        0.812224      1.000000
              predict_long  long return
predict_long      1.000000     0.846933
long return       0.846933     1.000000
